In [1]:
from dermclass_models2.preprocessing import TextPreprocessors, ImagePreprocessors, StructuredPreprocessor
from dermclass_models2.config import TextConfig, ImageConfig, StructuredConfig

In [2]:
# sp = StructuredPreprocessor(StructuredConfig)
# sp.load_data()

In [3]:
# ip = ImagePreprocessors(ImageConfig)
# ip.load_data()

In [6]:
tp = TextPreprocessors(TextConfig)
tp.load_data(use_generators=False)

2020-12-01 20:00:19,418 — dermclass_models2.preprocessing — INFO —_load_class_from_dir:272 — Successfully loaded class lichen_planus
2020-12-01 20:00:19,438 — dermclass_models2.preprocessing — INFO —_load_class_from_dir:272 — Successfully loaded class psoriasis
2020-12-01 20:00:19,442 — dermclass_models2.preprocessing — INFO —_load_data_from_files:285 — Successfully loaded the data
2020-12-01 20:00:19,444 — dermclass_models2.preprocessing — INFO —_split_target_structured:55 — Successfully splat the target data
2020-12-01 20:00:19,448 — dermclass_models2.preprocessing — INFO —_split_train_test_structured:78 — Successfully splat train and test data
2020-12-01 20:00:19,451 — dermclass_models2.preprocessing — INFO —_load_data_structured:85 — Successfully loaded the data


(                                                 text
 8   https://www.psoriasis.org/about-psoriasis/  Wh...
 5   https://www.webmd.com/skin-problems-and-treatm...
 2   https://dermnetnz.org/topics/lichen-planus/  W...
 1   https://www.nhs.uk/conditions/lichen-planus/  ...
 12  https://dermnetnz.org/topics/psoriasis/  What ...
 4   https://en.wikipedia.org/wiki/Lichen_planus  L...
 7   https://www.mayoclinic.org/diseases-conditions...
 10  https://www.nhs.uk/conditions/psoriasis/sympto...
 3   https://www.healthline.com/health/lichen-planu...
 6   https://en.wikipedia.org/wiki/Psoriasis#cite_n...,
                                                  text
 11  https://www.medicinenet.com/psoriasis/article....
 9   https://www.nhs.uk/conditions/psoriasis/  Psor...
 0   https://www.mayoclinic.org/diseases-conditions...,
 8         psoriasis
 5         psoriasis
 2     lichen_planus
 1     lichen_planus
 12        psoriasis
 4     lichen_planus
 7         psoriasis
 10        psoriasis
 3   